In [ ]:
#!pip install aquacrop==2.2
#!pip install numba==0.55
#!pip install statsmodels==0.13.2
#!pip install SALib
#!pip install -U scikit-learn scipy matplotlib
!pip install --upgrade pyswarm  #https://pythonhosted.org/pyswarm/

In [167]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse
from aquacrop.utils import prepare_weather, get_filepath
from aquacrop import AquaCropModel, Soil, Crop, InitialWaterContent, IrrigationManagement
#from aquacrop.entities import IrrigationManagement

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime
import pickle 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import sklearn.metrics as metrics
from pyswarm import pso
import os
from os import chdir, getcwd
import statistics as stats


In [3]:
os.chdir('/home/jupyter-wndlovu/') # change working directory

wd=getcwd()

In [83]:
def soilCompart(soilList):
    custom_soil = []
    for df in soilList:

        #soils_df = df.groupby(['Year'])
        soils_df = [v for k, v in df.groupby('Year')]
        #print(soils_df)
        for j in soils_df:
            #df_list = []
            soils_df_pivot = j.drop(['crop_mn_codeyear', 'Year'], axis=1)
            #print(soils_df_pivot)
            for i, row in soils_df_pivot.iterrows():
                #print(row)
                #print(soils_df_pivot)
                if row['depth'] == "0_5":
                    pred_thWP_5 = ((-0.024*((soils_df_pivot['sand'][i])/100))) + ((0.487*((soils_df_pivot['clay'][i])/100))) + ((0.006*((soils_df_pivot['om'][i])/100))) + ((0.005*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.013*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.068*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.031
                    wp_5 = pred_thWP_5 + (0.14 * pred_thWP_5) - 0.02
                    pred_thFC_5 = ((-0.251*((soils_df_pivot['sand'][i])/100))) + ((0.195*((soils_df_pivot['clay'][i])/100)))+ ((0.011*((soils_df_pivot['om'][i])/100))) + ((0.006*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.027*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.452*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.299
                    fc_5 = pred_thFC_5 + (1.283 * (np.power(pred_thFC_5, 2))) - (0.374 * pred_thFC_5) - 0.015
                        #fc = pred_thFC + (1.283 * (pred_thFC*pred_thFC)) - (0.374 * pred_thFC) - 0.015
                    ts_5 =soils_df_pivot["theta_s"][i]
                    ks_5=(soils_df_pivot['ksat'][i])*240
                if row['depth'] == "100_200":
                    pred_thWP_200 = ((-0.024*((soils_df_pivot['sand'][i])/100))) + ((0.487*((soils_df_pivot['clay'][i])/100))) + ((0.006*((soils_df_pivot['om'][i])/100))) + ((0.005*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.013*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.068*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.031
                    wp_200 = pred_thWP_200 + (0.14 * pred_thWP_200) - 0.02
                    pred_thFC_200 = ((-0.251*((soils_df_pivot['sand'][i])/100))) + ((0.195*((soils_df_pivot['clay'][i])/100)))+ ((0.011*((soils_df_pivot['om'][i])/100))) + ((0.006*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.027*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.452*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.299
                    fc_200 = pred_thFC_200 + (1.283 * (np.power(pred_thFC_200, 2))) - (0.374 * pred_thFC_200) - 0.015
                    #fc = pred_thFC + (1.283 * (pred_thFC*pred_thFC)) - (0.374 * pred_thFC) - 0.015
                    ts_200 =soils_df_pivot["theta_s"][i]
                    ks_200=(soils_df_pivot['ksat'][i])*240
                if row['depth'] == "15_30":
                    pred_thWP_30 = ((-0.024*((soils_df_pivot['sand'][i])/100))) + ((0.487*((soils_df_pivot['clay'][i])/100))) + ((0.006*((soils_df_pivot['om'][i])/100))) + ((0.005*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.013*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.068*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.031
                    wp_30 = pred_thWP_30 + (0.14 * pred_thWP_30) - 0.02
                    pred_thFC_30 = ((-0.251*((soils_df_pivot['sand'][i])/100))) + ((0.195*((soils_df_pivot['clay'][i])/100)))+ ((0.011*((soils_df_pivot['om'][i])/100))) + ((0.006*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.027*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.452*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.299
                    fc_30 = pred_thFC_30 + (1.283 * (np.power(pred_thFC_30, 2))) - (0.374 * pred_thFC_30) - 0.015
                    #fc = pred_thFC + (1.283 * (pred_thFC*pred_thFC)) - (0.374 * pred_thFC) - 0.015
                    ts_30 =soils_df_pivot["theta_s"][i]
                    ks_30=(soils_df_pivot['ksat'][i])*240
                if row['depth'] == "30_60":
                    pred_thWP_60 = ((-0.024*((soils_df_pivot['sand'][i])/100))) + ((0.487*((soils_df_pivot['clay'][i])/100))) + ((0.006*((soils_df_pivot['om'][i])/100))) + ((0.005*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.013*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.068*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.031
                    wp_60 = pred_thWP_60 + (0.14 * pred_thWP_60) - 0.02
                    pred_thFC_60 = ((-0.251*((soils_df_pivot['sand'][i])/100))) + ((0.195*((soils_df_pivot['clay'][i])/100)))+ ((0.011*((soils_df_pivot['om'][i])/100))) + ((0.006*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.027*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.452*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.299
                    fc_60 = pred_thFC_60 + (1.283 * (np.power(pred_thFC_60, 2))) - (0.374 * pred_thFC_60) - 0.015
                    #fc = pred_thFC + (1.283 * (pred_thFC*pred_thFC)) - (0.374 * pred_thFC) - 0.015
                    ts_60 =soils_df_pivot["theta_s"][i]
                    ks_60=(soils_df_pivot['ksat'][i])*240
                if row['depth'] == "5_15":
                    pred_thWP_15 = ((-0.024*((soils_df_pivot['sand'][i])/100))) + ((0.487*((soils_df_pivot['clay'][i])/100))) + ((0.006*((soils_df_pivot['om'][i])/100))) + ((0.005*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.013*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.068*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.031
                    wp_15 = pred_thWP_15 + (0.14 * pred_thWP_15) - 0.02
                    pred_thFC_15 = ((-0.251*((soils_df_pivot['sand'][i])/100))) + ((0.195*((soils_df_pivot['clay'][i])/100)))+ ((0.011*((soils_df_pivot['om'][i])/100))) + ((0.006*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.027*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.452*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.299
                    fc_15 = pred_thFC_15 + (1.283 * (np.power(pred_thFC_15, 2))) - (0.374 * pred_thFC_15) - 0.015
                    #fc = pred_thFC + (1.283 * (pred_thFC*pred_thFC)) - (0.374 * pred_thFC) - 0.015
                    ts_15 =soils_df_pivot["theta_s"][i]
                    #print(ts_15)
                    ks_15=(soils_df_pivot['ksat'][i])*240
                if row['depth'] == "60_100":
                    pred_thWP_100 = ((-0.024*((soils_df_pivot['sand'][i])/100))) + ((0.487*((soils_df_pivot['clay'][i])/100))) + ((0.006*((soils_df_pivot['om'][i])/100))) + ((0.005*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.013*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.068*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.031
                    wp_100 = pred_thWP_100 + (0.14 * pred_thWP_100) - 0.02
                    pred_thFC_100 = ((-0.251*((soils_df_pivot['sand'][i])/100))) + ((0.195*((soils_df_pivot['clay'][i])/100)))+ ((0.011*((soils_df_pivot['om'][i])/100))) + ((0.006*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.027*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.452*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.299
                    fc_100 = pred_thFC_100 + (1.283 * (np.power(pred_thFC_100, 2))) - (0.374 * pred_thFC_100) - 0.015
                    #fc = pred_thFC + (1.283 * (pred_thFC*pred_thFC)) - (0.374 * pred_thFC) - 0.015
                    ts_100 =soils_df_pivot["theta_s"][i]
                    ks_100=(soils_df_pivot['ksat'][i])*240



                    custom = Soil('custom',cn=46,rew=7, dz=[0.025]*2+[0.05]*2+[0.075]*2+[0.15]*2+[0.2]*2+[0.5]*2)

                    custom.add_layer(thickness=0.05,thS=ts_5, # assuming soil properties are the same in the upper 0.1m
                         Ksat=ks_5,thWP =wp_5 , 
                         thFC = fc_5, penetrability = 100.0)
                    custom.add_layer(thickness=0.15,thS=ts_15, # assuming soil properties are the same in the upper 0.1m
                                 Ksat=ks_15,thWP =wp_15 , 
                                 thFC = fc_15, penetrability = 100.0)
                    custom.add_layer(thickness=0.3,thS=ts_30, # assuming soil properties are the same in the upper 0.1m
                                 Ksat=ks_30,thWP =wp_30 , 
                                 thFC = fc_30, penetrability = 100.0)
                    custom.add_layer(thickness=0.3,thS=ts_60, # assuming soil properties are the same in the upper 0.1m
                                 Ksat=ks_60,thWP =wp_60 , 
                                 thFC = fc_60, penetrability = 100.0)
                    custom.add_layer(thickness=0.4,thS=ts_100, # assuming soil properties are the same in the upper 0.1m
                                 Ksat=ks_100,thWP =wp_100 , 
                                 thFC = fc_100, penetrability = 100.0)
                    custom.add_layer(thickness=1,thS=ts_200, # assuming soil properties are the same in the upper 0.1m
                                 Ksat=ks_200,thWP =wp_200 , 
                                 thFC = fc_200, penetrability = 100.0)
                    custom_soil.append(custom)
                    #df_list.append(custom)

                    #print(custom)

    return(custom_soil)

In [243]:
with open(wd + '/eggs/data/input_dict.pickle', 'rb') as input_data: 
    input_dict = pickle.load(input_data) 
    
corn_yield = pd.read_csv(wd + '/eggs/data/CornYield_GMD4_WNdlovu_v1_20230117.csv')
irrig_depth = pd.read_csv(wd + '/eggs/data/IrrigationDepth_GMD4_WNdlovu_v1_20230123.csv')

with open(wd + '/eggs/data/irrigated_corn_calibration_params.txt') as ic:
    # Read the contents of the file into a variable
    irrig_corn_calib = ic.readlines()
    
with open(wd + '/eggs/data/rainfed_corn_calibration_params.txt') as rc:
    # Read the contents of the file into a variable
    rainfed_corn_calib = rc.readlines()   

# Irrigated corn

In [151]:
irrig_corn = '1' # code for irrigated corn
irrig_corn_dict = {k:v for (k,v) in input_dict.items() if irrig_corn in k}

In [152]:
# calibrate using the decatur, gove, graham, logan, rawlins
calibration = list(irrig_corn_dict.items())[:5]
#calibration

In [154]:
# getting the different datasets
gridmet = [sublist[1][0] for sublist in calibration] # data stored in nested list with 0 having the 
lai = [sublist[1][1] for sublist in calibration]
soil = [sublist[1][2] for sublist in calibration]

### Soil data is currently being saved in one list - wondering if saving it in 5 lists make more sense (one list for each county)

In [116]:
soil_irrig = soilCompart(soil)

## Get usda yield for the calibration counties

In [226]:
#corn_yield = pd.read_csv(wd + '/eggs/data/CornYield_GMD4_WNdlovu_v1_20230117.csv')
corn_yield = corn_yield[corn_yield['Commodity'] == 'CORN']
irrig_usda = corn_yield[corn_yield['Irrig_status'] == 'irrigated']
rainfed_usda = corn_yield[corn_yield['Irrig_status'] == 'rainfed']

In [218]:
irrig_depth = irrig_depth[irrig_depth['gmd'] == 4]

## WIMAS irrigation depth

In [ ]:
# create full county name variable
irrig_depth['County'] = np.where(
         irrig_depth['county_abrev'] == 'WA', 'WALLACE',
         np.where(
              irrig_depth['county_abrev'] == 'RA', 'RAWLINS',
              np.where(
                   irrig_depth['county_abrev'] == 'DC', 'DECATUR',
                   np.where(
                        irrig_depth['county_abrev'] == 'GH', 'GRAHAM',
                        np.where(
                             irrig_depth['county_abrev'] == 'CN', 'CHEYENNE',
                             np.where(
                                 irrig_depth['county_abrev'] == 'LG', 'LOGAN',
                             np.where(
                                         irrig_depth['county_abrev'] == 'SD', 'SHERIDAN',
                                     np.where(
                                          irrig_depth['county_abrev'] == 'TH', 'THOMAS',
                                          np.where(
                                               irrig_depth['county_abrev'] == 'SH', 'SHERMAN', 'GOVE'
                                          )  
                                       )

                                  )
                             )
                        )
                   )
              )
         )
    )

# rename year column
irrig_depth = irrig_depth.rename(columns = {'WUA_YEAR': 'Year'})

In [228]:
irrig_depth

,Unnamed: 0,PDIV_ID,long_nad83,lat_nad83,gmd,county_abrev,hpa_region,Year,crop_code,crop_name,irrig_depth,County
1,1,11,-101.7118,39.55035,4.0,SH,yes,1990,2,Corn,610.252907,SHERMAN
2,2,142,-101.8833,39.20584,4.0,SH,yes,1990,2,Corn,417.878340,SHERMAN
3,3,148,-100.7909,39.05746,4.0,GO,yes,1990,2,Corn,488.024600,GOVE
4,4,196,-101.1394,39.28588,4.0,TH,yes,1990,2,Corn,797.694169,THOMAS
6,6,245,-100.9127,39.31441,4.0,TH,yes,1990,2,Corn,539.747969,THOMAS
...,...,...,...,...,...,...,...,...,...,...,...,...
80651,80651,68931,-100.7064,39.33598,4.0,SD,yes,2021,4,Soybeans,176.980645,SHERIDAN
80657,80657,74305,-100.5955,39.42278,4.0,SD,yes,2021,4,Soybeans,277.541463,SHERIDAN
80669,80669,84606,-101.2103,39.55463,4.0,TH,yes,2021,4,Soybeans,301.657732,THOMAS
80671,80671,85188,-100.0695,39.29602,4.0,GH,yes,2021,4,Soybeans,349.250000,GRAHAM


In [262]:
stages = [15, 42, 131, 91, 83, 67, 60, 13.7, 1.31]
pheno_df = pd.DataFrame(stages)
pheno_df = pheno_df.transpose()
pheno_df.columns = ['Emergence', 'Senescence', 'Maturity', 'MaxRooting', 'HIstart', 'Flowering', 'YldForm', 'CGC', 'CDC']


In [265]:
def objFun(default_params):
    default_params = default_params
    
    Emergence = default_params[['Emergence']],
    Senescence = default_params[['Senescence']],
    Maturity = default_params[['Maturity']],
    MaxRooting = default_params[['MaxRooting']],
    HIstart = default_params[['HIstart']],
    Flowering = default_params[['Flowering']],
    YldForm = default_params[['YldForm']],
                
                # canopy cover profile
    CGC = default_params[['CGC']],
    CDC = default_params[['CDC']]
    print(CDC)
    


In [266]:
objFun(pheno_df)

    CDC
0  1.31


In [250]:
irrig_corn_calib = [item.strip() for item in irrig_corn_calib]
irrig_corn_calib

['ccx', 'kc', 'wp', 'wpy', 'hi', 'kcdcl', 'rtm', 'smt']

In [ ]:
def objFun(calibration_params, crop_name, planting, default_params):
    
    wpy, ccx, wp, kc, smt, rtm, kcdcl = calibration_params
    
    = default_params
    
    irr_mngt = IrrigationManagement(irrigation_method=1,SMT=[smt]*4) 
    
    crop_calibrated = Crop("crop", planting_date='05/01',
                # growth stages
                CalendarType = 1,
                SwitchGDD = 1,  #Convert calendar to gdd mode if inputs are given in calendar days (0 = No; 1 = Yes)
                ##Crop Development
                Emergence = 15, #15
                Senescence = 42, #42
                Maturity = 131, #131
                MaxRooting = 91, #91
                HIstart = 83,
                Flowering = 67,
                YldForm = 60,
                Zmin = rtm,
                
                # canopy cover profile
                CCx = ccx,
                CGC = 13.7,
                CDC = 1.31,
                
                
                WPy = wpy,
                Kcb = kc,
                fage = kcdcl,         
                WP = wp,                  
        )


    # run model
    model_c = AquaCropModel(sim_start,sim_end,wdf,custom,crop_calibrated,initWC, irr_mngt)
    model_c.run_model(till_termination=True) # run model till the end
    model_c_et = model_c._outputs.water_flux
    model_c_irr = model_c._outputs.final_stats
    #model_c_water_storage = model_c._outputs.water_storage
    #model_c_crp_grwth = model_c._outputs.crop_growth

     # make year a column 

In [ ]:
def objFun(x, crop):
    
    wpy, ccx, wp, kc, smt, rtm, kcdcl = x
    
    irr_mngt = IrrigationManagement(irrigation_method=1,SMT=[smt]*4) 
    
    crop_calibrated = Crop("crop", planting_date='05/01',
                # growth stages
                CalendarType = 1,
                SwitchGDD = 1,  #Convert calendar to gdd mode if inputs are given in calendar days (0 = No; 1 = Yes)
                ##Crop Development
                Emergence = 15, #15
                Senescence = 42, #42
                Maturity = 131, #131
                MaxRooting = 91, #91
                HIstart = 83,
                Flowering = 67,
                YldForm = 60,
                Zmin = rtm,
                
                # canopy cover profile
                CCx = ccx,
                CGC = 13.7,
                CDC = 1.31,
                
                
                WPy = wpy,
                Kcb = kc,
                fage = kcdcl,         
                WP = wp,                  
        )


    # run model
    model_c = AquaCropModel(sim_start,sim_end,wdf,custom,crop_calibrated,initWC, irr_mngt)
    model_c.run_model(till_termination=True) # run model till the end
    model_c_et = model_c._outputs.water_flux
    model_c_irr = model_c._outputs.final_stats
    #model_c_water_storage = model_c._outputs.water_storage
    #model_c_crp_grwth = model_c._outputs.crop_growth

     # make year a column 
    model_c_irr = model_c_irr.assign(Year =  model_c_irr['Harvest Date (YYYY/MM/DD)'].dt.year)
    model_c_irr = model_c_irr.rename(columns={
                       'Yield (tonne/ha)': 'Calib Yield (t/ha)',
                       'Seasonal irrigation (mm)': 'Calib Irrigation (mm)'
                       })

    model_c_irr = model_c_irr[['Year', 'Calib Yield (t/ha)', 'Calib Irrigation (mm)']]

    #return model_c_irr

    # compare models
    model_comp = model_c_irr.merge(sher_irrig_corn,on='Year').merge(sher_wimas, on = 'Year')
    model_comp = model_comp.assign(YieldUSDA = model_comp['Value']*0.0673)
    
   # return(model_comp)
    
    y_yield = model_comp[['YieldUSDA']]
    yhat_yield = model_comp[['Calib Yield (t/ha)']]
    
    y_irrig = model_comp[['irrig_depth']]
    yhat_irrig = model_comp[['Calib Irrigation (mm)']]
    
    #print(y)
    #return(yhat)
    
    var_yield = y_yield.var()
    var_irrig = y_irrig.var()
    
    #return var_yield, var_irrig
    #fitness = ((y[['YieldUSDA']]-yhat[['Calib Yield (t/ha)']])**2).sum() # Calculate fitness score based on AquaCrop performance
    #fitness = (((np.array(y_yield)-np.array(yhat_yield))**2)*(1/var_yield)).sum() #+ (var_irrig*((np.array(y_irrig)-np.array(yhat_irrig))**2)).sum().item()
    fitness = (((np.array(y_yield)-np.array(yhat_yield))**2)*np.array(1/var_yield)).sum().item() +(((np.array(y_irrig)-np.array(yhat_irrig))**2)*np.array(1/var_irrig)).sum().item()
    return fitness # fitness 



In [ ]:
dec_met = gridmet[0]
dec_met

In [ ]:
gridmet_df = [v for k, v in dec_met.groupby('Year')]

for met in gridmet_df:
    yr = met.Year.unique()
    print(yr)

In [185]:
calib_counties = []
for county in gridmet:
    model_results = []
    gridmet_df = [v for k, v in county.groupby('Year')]
    for met in gridmet_df:
        county = met['crop_mn_codeyear'].str.replace(r'^.*?(?=[a-z, A-Z])', '', regex=True).unique()[0].upper()
        #print(county)
        yr = met['Year'].unique()[0]
        wdf = met[['MinTemp', 
                             'MaxTemp',
                             'Precipitation', 
                             'ReferenceET',
                             'Date'
                            ]]
        sim_start = f'{yr}/01/01' #dates to match crop data
        sim_end = f'{yr}/12/31'
        #print(sim_start)
        initWC = InitialWaterContent(value=['FC'])
        irr_mngt = IrrigationManagement(irrigation_method=1,SMT=[80]*4) # no irrigation

        # uncalibrated model - yield
        crop_uncalibrated = Crop('Maize', planting_date='05/01') 

        # run model
        model_uc = AquaCropModel(sim_start,sim_end,wdf,soil_irrig[0],crop_uncalibrated,initWC, irr_mngt)
        model_uc.run_model(till_termination=True) # run model till the end
        #model_uc_et = model_uc._outputs.water_flux
        model_uc_irr = model_uc._outputs.final_stats
        model_results.append(model_uc_irr)

    model_results_df = pd.concat(model_results)
    model_results_df['COUNTY'] = county
    
    calib_counties.append(model_results_df)




In [178]:
met['crop_mn_codeyear'].str.replace(r'^.*?(?=[a-z, A-Z])', '', regex=True)

315858    Rawlins
315925    Rawlins
315992    Rawlins
316059    Rawlins
316126    Rawlins
           ...   
340045    Rawlins
340112    Rawlins
340179    Rawlins
340246    Rawlins
340313    Rawlins
Name: crop_mn_codeyear, Length: 366, dtype: object

In [186]:
model_results_df

,Season,crop Type,Harvest Date (YYYY/MM/DD),Harvest Date (Step),Yield (tonne/ha),Seasonal irrigation (mm),COUNTY
0,0,Maize,2006-09-10,251,14.133846,737.049274,RAWLINS
0,0,Maize,2007-09-10,251,14.222279,643.980420,RAWLINS
0,0,Maize,2008-09-10,252,14.179405,627.624020,RAWLINS
0,0,Maize,2009-09-10,251,14.000727,497.795997,RAWLINS
0,0,Maize,2010-09-10,251,14.382001,664.499176,RAWLINS
0,0,Maize,2011-09-10,251,14.365267,669.539784,RAWLINS
0,0,Maize,2012-09-10,252,13.696813,974.315540,RAWLINS
0,0,Maize,2013-09-10,251,14.059590,760.307366,RAWLINS
0,0,Maize,2014-09-10,251,14.219363,604.816964,RAWLINS
0,0,Maize,2015-09-10,251,14.115834,581.390246,RAWLINS


In [ ]:

sim_start = f'{1979}/01/01' #dates to match crop data
sim_end = f'{1979}/12/31'
initWC = InitialWaterContent(value=['FC'])
irr_mngt = IrrigationManagement(irrigation_method=1,SMT=[80]*4) # no irrigation


# uncalibrated model - yield
crop_uncalibrated = Crop('Maize', planting_date='05/01') 

# run model
model_uc = AquaCropModel(sim_start,sim_end,wdf,custom,crop_uncalibrated,initWC, irr_mngt)
model_uc.run_model(till_termination=True) # run model till the end
#model_uc_et = model_uc._outputs.water_flux
model_uc_irr = model_uc._outputs.final_stats
#model_df_water_storage = model._outputs.water_storage
#model_df_crp_grwth = model._outputs.crop_growth


In [ ]:
sim_start = '2000/01/01' #dates to match crop data
sim_end = '2020/12/31'
initWC = InitialWaterContent(value=['FC'])
irr_mngt = IrrigationManagement(irrigation_method=1,SMT=[80]*4) # no irrigation


# uncalibrated model - yield
crop_uncalibrated = Crop('Maize', planting_date='05/01') 

# run model
model_uc = AquaCropModel(sim_start,sim_end,wdf,custom,crop_uncalibrated,initWC, irr_mngt)
model_uc.run_model(till_termination=True) # run model till the end
#model_uc_et = model_uc._outputs.water_flux
model_uc_irr = model_uc._outputs.final_stats
#model_df_water_storage = model._outputs.water_storage
#model_df_crp_grwth = model._outputs.crop_growth

## Create year variable
model_uc_irr = model_uc_irr.assign(Year =  model_uc_irr['Harvest Date (YYYY/MM/DD)'].dt.year)
model_uc_irr = model_uc_irr.rename(columns={
                   'Yield (tonne/ha)': 'Uncalib Yield (t/ha)',
                   'Seasonal irrigation (mm)': 'Uncalib Irrigation (mm)'
                   })
model_uc_irr = model_uc_irr[['Year', 'Uncalib Yield (t/ha)', 'Uncalib Irrigation (mm)']]
